# Coin Metrics - Market Data Feed
## > OPTION CHAIN EXAMPLE

### Notebook configuration:

In [1]:
import os
from os import environ
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import logging
from datetime import date, datetime, timedelta
from coinmetrics.api_client import CoinMetricsClient
import json
import logging
from pytz import timezone as timezone_conv
from datetime import timezone as timezone_info
import matplotlib.ticker as mticker
from matplotlib.ticker import ScalarFormatter
import matplotlib.pyplot as plt
%matplotlib inline

from pathlib import Path  

sns.set_theme()
sns.set(rc={'figure.figsize':(12,8)})

In [2]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [3]:
# We recommend privately storing your API key in your local environment.
try:
    api_key = environ["CM_API_KEY"]
    logging.info("Using API key found in environment")
except KeyError:
    api_key = ""
    logging.info("API key not found. Using community client")

client = CoinMetricsClient(api_key)

2022-03-30 11:19:54 INFO     API key not found. Using community client


## Pulling options quotes:

In [4]:
deribit_quotes = client.get_market_quotes(
    markets='deribit-*-option',
    limit_per_market=1
).to_dataframe()

In [5]:
deribit_quotes

,market,time,coin_metrics_id,ask_price,ask_size,bid_price,bid_size
0,deribit-BTC-15APR22-25000-C-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.5085,22.1,0.439,21.2
1,deribit-BTC-15APR22-25000-P-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0005,22.5,0.0,0.0
2,deribit-BTC-15APR22-30000-C-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.443,0.3,0.303,8.0
3,deribit-BTC-15APR22-30000-P-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0005,0.5,0.0,0.0
4,deribit-BTC-15APR22-34000-C-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.359,0.3,0.219,22.8
...,...,...,...,...,...,...,...
931,deribit-ETH-8APR22-4000-P-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.59,5.0,0.0,0.0
932,deribit-ETH-8APR22-4200-C-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.001,100.0,0.0005,914.0
933,deribit-ETH-8APR22-4200-P-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.6485,5.0,0.0,0.0
934,deribit-ETH-8APR22-4500-C-option,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0005,232.0,0.0,0.0


### Select an asset ticker:

In [6]:
asset='BTC'

### Extract the asset ticker, expiry date, and the option type (call/put) from contract names:


In [7]:
split = deribit_quotes["market"].str.split("-")[0:]

In [8]:
split

0      [deribit, BTC, 15APR22, 25000, C, option]
1      [deribit, BTC, 15APR22, 25000, P, option]
2      [deribit, BTC, 15APR22, 30000, C, option]
3      [deribit, BTC, 15APR22, 30000, P, option]
4      [deribit, BTC, 15APR22, 34000, C, option]
                         ...                    
931      [deribit, ETH, 8APR22, 4000, P, option]
932      [deribit, ETH, 8APR22, 4200, C, option]
933      [deribit, ETH, 8APR22, 4200, P, option]
934      [deribit, ETH, 8APR22, 4500, C, option]
935      [deribit, ETH, 8APR22, 4500, P, option]
Name: market, Length: 936, dtype: object

In [9]:
split_expiry = pd.DataFrame(datetime.strptime(i[2],'%d%b%y').strftime('%Y-%m-%d') for i in split)

In [10]:
split_asset = pd.DataFrame([i[1] for i in split])
split_strike = pd.DataFrame([i[3] for i in split])
split_side = pd.DataFrame([i[4] for i in split])

### Remove the 'C-Option' and 'P-Option' suffixes to isolate expiration date and strike price:

In [11]:
split_market = pd.DataFrame((i[0:4] for i in split))

split_market['market'] = (
    split_market[0].astype(str) + 
    '-' + split_market[1].astype(str) + 
    '-' + split_market[2].astype(str) + 
    '-' + split_market[3].astype(str) )

In [12]:
split_market['market']

0      deribit-BTC-15APR22-25000
1      deribit-BTC-15APR22-25000
2      deribit-BTC-15APR22-30000
3      deribit-BTC-15APR22-30000
4      deribit-BTC-15APR22-34000
                 ...            
931      deribit-ETH-8APR22-4000
932      deribit-ETH-8APR22-4200
933      deribit-ETH-8APR22-4200
934      deribit-ETH-8APR22-4500
935      deribit-ETH-8APR22-4500
Name: market, Length: 936, dtype: object

### Create new columns with categorical variables:

In [13]:
deribit_quotes['BTC'] = split_asset[0].isin([asset])

In [14]:
deribit_quotes['Call'] = split_side[0].isin(['C'])
deribit_quotes['Put'] = split_side[0].isin(['P'])
deribit_quotes['market'] = split_market['market']
deribit_quotes['strike'] = split_strike[:].astype(int)
deribit_quotes['expiry'] = split_expiry[:]

In [15]:
deribit_quotes

,market,time,coin_metrics_id,ask_price,ask_size,bid_price,bid_size,BTC,Call,Put,strike,expiry
0,deribit-BTC-15APR22-25000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.5085,22.1,0.439,21.2,True,True,False,25000,2022-04-15
1,deribit-BTC-15APR22-25000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0005,22.5,0.0,0.0,True,False,True,25000,2022-04-15
2,deribit-BTC-15APR22-30000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.443,0.3,0.303,8.0,True,True,False,30000,2022-04-15
3,deribit-BTC-15APR22-30000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0005,0.5,0.0,0.0,True,False,True,30000,2022-04-15
4,deribit-BTC-15APR22-34000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.359,0.3,0.219,22.8,True,True,False,34000,2022-04-15
...,...,...,...,...,...,...,...,...,...,...,...,...
931,deribit-ETH-8APR22-4000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.59,5.0,0.0,0.0,False,False,True,4000,2022-04-08
932,deribit-ETH-8APR22-4200,2022-03-30 15:19:00+00:00,1648653540000000-0,0.001,100.0,0.0005,914.0,False,True,False,4200,2022-04-08
933,deribit-ETH-8APR22-4200,2022-03-30 15:19:00+00:00,1648653540000000-0,0.6485,5.0,0.0,0.0,False,False,True,4200,2022-04-08
934,deribit-ETH-8APR22-4500,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0005,232.0,0.0,0.0,False,True,False,4500,2022-04-08


###  Isolate Bitcoin markets:

In [16]:
btc_quotes = deribit_quotes.loc[deribit_quotes['BTC'] == True]

### Sort by expiry and strike:

In [17]:
btc_quotes = btc_quotes.sort_values(by=["expiry","strike"],ascending=[True, True])

In [18]:
btc_quotes

,market,time,coin_metrics_id,ask_price,ask_size,bid_price,bid_size,BTC,Call,Put,strike,expiry
270,deribit-BTC-30MAR22-38000,2022-03-30 08:00:00+00:00,1648627200000000-0,0.0,0.0,0.0,0.0,True,True,False,38000,2022-03-30
271,deribit-BTC-30MAR22-38000,2022-03-30 08:00:00+00:00,1648627200000000-0,0.0,0.0,0.0,0.0,True,False,True,38000,2022-03-30
272,deribit-BTC-30MAR22-40000,2022-03-30 08:00:00+00:00,1648627200000000-0,0.0,0.0,0.0,0.0,True,True,False,40000,2022-03-30
273,deribit-BTC-30MAR22-40000,2022-03-30 08:00:00+00:00,1648627200000000-0,0.0,0.0,0.0,0.0,True,False,True,40000,2022-03-30
274,deribit-BTC-30MAR22-41000,2022-03-30 08:00:00+00:00,1648627200000000-0,0.0,0.0,0.0,0.0,True,True,False,41000,2022-03-30
...,...,...,...,...,...,...,...,...,...,...,...,...
387,deribit-BTC-31MAR23-200000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0,0.0,0.0,0.0,True,False,True,200000,2023-03-31
388,deribit-BTC-31MAR23-250000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.009,19.3,0.0065,9.7,True,True,False,250000,2023-03-31
389,deribit-BTC-31MAR23-250000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.0,0.0,0.0,0.0,True,False,True,250000,2023-03-31
392,deribit-BTC-31MAR23-300000,2022-03-30 15:19:00+00:00,1648653540000000-0,0.005,0.2,0.003,11.8,True,True,False,300000,2023-03-31


### Drop unnecessary columns:

In [19]:
btc_quotes = btc_quotes.drop(columns=['coin_metrics_id', 'time'])

### Create separate dataframes for Calls and Puts:

In [20]:
btc_calls = btc_quotes.loc[btc_quotes['Call'] == True]
btc_puts = btc_quotes.loc[btc_quotes['Put'] == True]

### Reformat Calls dataframe and drop duplicate markets:

In [21]:
btc_calls = btc_calls.rename(columns={"ask_price": "Call - Ask Price", 
                                      "ask_size": "Call - Ask Size", 
                                      "bid_price": "Call - Bid Price", 
                                      "bid_size": "Call - Bid Size", }).drop(columns=['BTC','Call','Put','expiry','strike'])
btc_calls = btc_calls[~btc_calls.market.duplicated(keep='first')]
btc_calls

,market,Call - Ask Price,Call - Ask Size,Call - Bid Price,Call - Bid Size
270,deribit-BTC-30MAR22-38000,0.0,0.0,0.0,0.0
272,deribit-BTC-30MAR22-40000,0.0,0.0,0.0,0.0
274,deribit-BTC-30MAR22-41000,0.0,0.0,0.0,0.0
276,deribit-BTC-30MAR22-42000,0.0,0.0,0.0,0.0
278,deribit-BTC-30MAR22-43000,0.0,0.0,0.0,0.0
...,...,...,...,...,...
380,deribit-BTC-31MAR23-140000,0.0335,31.0,0.028,18.8
382,deribit-BTC-31MAR23-150000,0.028,9.5,0.0255,1.0
386,deribit-BTC-31MAR23-200000,0.0145,23.2,0.0125,5.0
388,deribit-BTC-31MAR23-250000,0.009,19.3,0.0065,9.7


### Reformat Puts dataframe and drop duplicate markets:

In [22]:
btc_puts = btc_puts.rename(columns={"ask_price": "Put - Ask Price", 
                                    "ask_size": "Put - Ask Size", 
                                    "bid_price": "Put - Bid Price", 
                                    "bid_size": "Put - Bid Size", }).drop(columns=['BTC','Call','Put','expiry','strike'])
btc_puts = btc_puts[~btc_puts.market.duplicated(keep='last')]
btc_puts

,market,Put - Ask Price,Put - Ask Size,Put - Bid Price,Put - Bid Size
271,deribit-BTC-30MAR22-38000,0.0,0.0,0.0,0.0
273,deribit-BTC-30MAR22-40000,0.0,0.0,0.0,0.0
275,deribit-BTC-30MAR22-41000,0.0,0.0,0.0,0.0
277,deribit-BTC-30MAR22-42000,0.0,0.0,0.0,0.0
279,deribit-BTC-30MAR22-43000,0.0,0.0,0.0,0.0
...,...,...,...,...,...
381,deribit-BTC-31MAR23-140000,0.0,0.0,0.0,0.0
383,deribit-BTC-31MAR23-150000,0.0,0.0,0.0,0.0
387,deribit-BTC-31MAR23-200000,0.0,0.0,0.0,0.0
389,deribit-BTC-31MAR23-250000,0.0,0.0,0.0,0.0


### Merge Calls and Puts into unified Option Chain:

In [23]:
option_chain = pd.merge(btc_calls,btc_puts,on='market',validate='m:1')
option_chain = option_chain.set_index('market')
option_chain

,Call - Ask Price,Call - Ask Size,Call - Bid Price,Call - Bid Size,Put - Ask Price,Put - Ask Size,Put - Bid Price,Put - Bid Size
market,,,,,,,,
deribit-BTC-30MAR22-38000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
deribit-BTC-30MAR22-40000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
deribit-BTC-30MAR22-41000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
deribit-BTC-30MAR22-42000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
deribit-BTC-30MAR22-43000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
deribit-BTC-31MAR23-140000,0.0335,31.0,0.028,18.8,0.0,0.0,0.0,0.0
deribit-BTC-31MAR23-150000,0.028,9.5,0.0255,1.0,0.0,0.0,0.0,0.0
deribit-BTC-31MAR23-200000,0.0145,23.2,0.0125,5.0,0.0,0.0,0.0,0.0


### Export to CSV:

In [24]:
filepath = Path('./' + asset + '-option-chain.csv')  
option_chain.to_csv(filepath)